# plots

> Bayesian model trace plots

In [ ]:
#| default_exp plots

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti
import altair as alt
import arviz as az
import functools
import numpy as np
import operator
import pandas as pd
import scipy
import xarray as xr

In [ ]:
#| exporti
def chart_selection_vars(data_var):
    dims = list(data_var.dims[2:])

    if len(dims) > 0:
        return [alt.selection_point(fields=['chain', 'category'])]
    else:
        return [alt.selection_point(fields=['chain'])]

def legend_selection_vars(data_var):
    dims = list(data_var.dims[2:])

    if len(dims) > 0:
        return [
            alt.selection_point(fields=['chain'], bind='legend'),
            alt.selection_point(fields=['category'], bind='legend'),
        ]
    else:
        return [alt.selection_point(fields=['chain'], bind='legend')]

def limit_traces(data_var, category_limit=50, **_kwargs):
    dims = list(data_var.dims[2:])

    dim_sizes = [data_var.coords[dim].size for dim in dims]
    num_chains = data_var.coords['chain'].size
    num_groups = np.prod(dim_sizes) * num_chains

    if num_groups > category_limit:
        selected_dim_sizes = [1 for _ in dims]
        selected_dim_sizes[-1] = min(dim_sizes[-1], category_limit)
        current_dim_idx = len(dim_sizes) - 1

        while np.prod(selected_dim_sizes) * num_chains < category_limit:
            if selected_dim_sizes[current_dim_idx] < dim_sizes[current_dim_idx]:
                selected_dim_sizes[current_dim_idx] += 1
            else:
                current_dim_idx -= 1

        data_var = data_var.isel({dim: list(range(size)) for dim, size in zip(dims, selected_dim_sizes)})

    return data_var

def transform_density(data_var, num_density_points=50, **_kwargs):
    # https://docs.xarray.dev/en/stable/examples/apply_ufunc_vectorize_1d.html
    def gaussian_kde(data):
        x_pts = np.linspace(np.min(data), np.max(data), num_density_points)
        gkde = scipy.stats.gaussian_kde(data)
        return gkde.evaluate(x_pts) / 100., x_pts

    density_data, x_pts_data = xr.apply_ufunc(
        gaussian_kde,
        data_var,
        input_core_dims=[['draw']],
        output_core_dims=[['_n'], ['_n']],
        exclude_dims=set(('draw',)),
        vectorize=True,
    )

    transformed_data = xr.merge([density_data.rename('density'), x_pts_data.rename(data_var.name)])
    return transformed_data.to_dataframe().reset_index(['_n'], drop=True)

def transform_stats(data_var, **_kwargs):
    dims = list(data_var.dims[2:])

    def merge_stats(df, stats, stat_name, dims):
        if len(dims) > 0:
            return pd.merge(
                df,
                stats.to_dataframe().reset_index().rename(columns={data_var.name: stat_name}),
                how='left',
                left_on=dims,
                right_on=dims
            )
        else:
            return df.assign(**{stat_name: stats[data_var.name].to_numpy()})

    df = data_var.mean(['draw']).to_dataframe().reset_index().rename(columns={data_var.name: 'mean'})
    df = merge_stats(df, data_var.median(['draw']), 'median', ['chain'] + dims)
    df = merge_stats(df, data_var.std(['draw']), 'sd', ['chain'] + dims)
    df = merge_stats(df, az.rhat(data_var), 'r_hat', dims)
    df = merge_stats(df, az.ess(data_var, method='bulk'), 'ess_bulk', dims)

    return df.set_index(['chain'] + dims)

def transform_rank(data_var, num_rank_bins=None, **_kwargs):
    # Twice the result of Sturges' formula
    if num_rank_bins is None:
        num_rank_bins = int(np.ceil(2. * np.log2(data_var.coords['draw'].size)) + 1)

    # https://docs.xarray.dev/en/stable/examples/apply_ufunc_vectorize_1d.html
    def trace_rank(data):
        ranks = scipy.stats.rankdata(data, method='average')
        return ranks.reshape(data.shape)

    rank_data = xr.apply_ufunc(
        trace_rank,
        data_var,
        input_core_dims=[['chain', 'draw']],
        output_core_dims=[['chain', 'draw']],
        vectorize=True,
    )

    num_ranks = data_var.coords['chain'].size * data_var.coords['draw'].size

    def rank_count(data):
        # Note: arviz.plot_rank uses range=(0, num_ranks). However, the lowest rank is 1.
        hist, bin_edges = np.histogram(data, bins=num_rank_bins, range=(1, num_ranks))
        hist_freq = hist / hist.sum()
        return hist_freq, (bin_edges[1:] + bin_edges[:-1]) / 2

    rank_hist_data, rank_bins_data = xr.apply_ufunc(
        rank_count,
        rank_data,
        input_core_dims=[['draw']],
        output_core_dims=[['bin'], ['bin']],
        exclude_dims=set(('draw',)),
        vectorize=True,
    )

    transformed_data = xr.merge([rank_hist_data.rename('mass'), rank_bins_data.rename('rank')])

    return transformed_data.to_dataframe()

In [ ]:
#| export
def plot_density(data_var, legend=None, chart_selection=None, legend_selection=None, properties={}, **kwargs):
    legend = alt.Legend(symbolLimit=10) if legend is None else legend
    chart_selection = chart_selection_vars(data_var) if chart_selection is None else chart_selection
    legend_selection = legend_selection_vars(data_var) if legend_selection is None else legend_selection

    dims = list(data_var.dims[2:])
    data_var = limit_traces(data_var, **kwargs)
    density_df = transform_density(data_var, **kwargs)
    stats_df = transform_stats(data_var, **kwargs)
    chart_df = density_df.join(stats_df).reset_index()

    tooltip = [
        alt.Tooltip(f'{data_var.name}', format='.2f'),
        alt.Tooltip('density', format='.2%'),
        alt.Tooltip('chain:O'),
    ]

    stats_tooltip = [
        alt.Tooltip('mean:Q', format='.2f'),
        alt.Tooltip('median:Q', format='.2f'),
        alt.Tooltip('sd:Q', format='.2f'),
        alt.Tooltip('ess_bulk:Q', format='.0f'),
        alt.Tooltip('r_hat:Q', format='.3f'),
    ]

    selection_filter_pred = functools.reduce(operator.and_, legend_selection)
    selection_opacity_pred = functools.reduce(operator.and_, legend_selection + chart_selection)

    encode_kwargs = {
        'strokeDash': alt.StrokeDash('chain:O', legend=legend),
        'opacity': alt.condition(selection_opacity_pred, alt.value(0.9), alt.value(0.1)),
    }

    if len(dims) > 0:
        category_title = ' / '.join(dims)
        chart_df['category'] = chart_df[dims].apply(lambda x: ' / '.join(x.astype(str)), axis=1)
        tooltip += [alt.Tooltip('category:N', title=category_title)]
        encode_kwargs['color'] = alt.Color('category:N', legend=legend).title(category_title)

    chart_base = alt.Chart(chart_df)

    chart_line = chart_base.mark_line().encode(
        x=alt.X(f'{data_var.name}:Q'),
        y=alt.Y('density:Q', axis=alt.Axis(format="%")),
        **encode_kwargs,
    ).add_params(
        *(legend_selection + chart_selection)
    )

    chart_hover = alt.selection_point(
        fields=[f'{data_var.name}'],
        nearest=True,
        on='pointerover',
        empty=False,
        toggle=False,
        clear='pointerout',
    )

    chart_point = chart_line.transform_filter(selection_filter_pred).transform_filter(chart_hover).mark_circle(size=65)

    chart_tooltip = chart_base.mark_rule().encode(
        x=alt.X(f'{data_var.name}:Q'),
        y=alt.Y('density:Q', axis=alt.Axis(format="%")),
        opacity=alt.condition(chart_hover, alt.value(0.3), alt.value(0)),
        tooltip=tooltip + stats_tooltip,
    ).transform_filter(
        selection_filter_pred
    ).add_params(
        chart_hover
    )

    return (chart_line + chart_point + chart_tooltip).properties(**properties)

def plot_rank(data_var, legend=None, chart_selection=None, legend_selection=None, properties={}, **kwargs):
    legend = alt.Legend(symbolLimit=10) if legend is None else legend
    chart_selection = chart_selection_vars(data_var) if chart_selection is None else chart_selection
    legend_selection = legend_selection_vars(data_var) if legend_selection is None else legend_selection

    dims = list(data_var.dims[2:])
    data_var = limit_traces(data_var, **kwargs)
    trace_rank_df = transform_rank(data_var, **kwargs)
    stats_df = transform_stats(data_var, **kwargs)
    chart_df = trace_rank_df.join(stats_df).reset_index()

    tooltip = [
        alt.Tooltip('rank:Q', format='.0f'),
        alt.Tooltip('mass', format='.2%'),
        alt.Tooltip('chain:O'),
    ]

    stats_tooltip = [
        alt.Tooltip('mean:Q', format='.2f'),
        alt.Tooltip('median:Q', format='.2f'),
        alt.Tooltip('sd:Q', format='.2f'),
        alt.Tooltip('ess_bulk:Q', format='.0f'),
        alt.Tooltip('r_hat:Q', format='.3f'),
    ]

    selection_filter_pred = functools.reduce(operator.and_, legend_selection)
    selection_opacity_pred = functools.reduce(operator.and_, legend_selection + chart_selection)

    encode_kwargs = {
        'strokeDash': alt.StrokeDash('chain:O', legend=legend),
        'opacity': alt.condition(selection_opacity_pred, alt.value(0.9), alt.value(0.05)),
    }

    if len(dims) > 0:
        category_title = ' / '.join(dims)
        chart_df['category'] = chart_df[dims].apply(lambda x: ' / '.join(x.astype(str)), axis=1)
        tooltip += [alt.Tooltip('category:N', title=category_title)]
        encode_kwargs['color'] = alt.Color('category:N', legend=legend).title(category_title)

    chart_base = alt.Chart(chart_df)

    chart_line = chart_base.mark_line(interpolate='step').encode(
        x=alt.X('rank:Q'),
        y=alt.Y('mass:Q', axis=alt.Axis(format="%")),
        **encode_kwargs,
    ).add_params(
        *(legend_selection + chart_selection)
    )

    chart_hover = alt.selection_point(
        fields=['rank'],
        nearest=True,
        on='pointerover',
        empty=False,
        toggle=False,
        clear='pointerout',
    )

    chart_point = chart_line.transform_filter(selection_filter_pred).transform_filter(chart_hover).mark_circle(size=65)

    chart_tooltip = chart_base.mark_rule().encode(
        x=alt.X('rank:Q'),
        y=alt.Y('mass:Q', axis=alt.Axis(format="%")),
        opacity=alt.condition(chart_hover, alt.value(0.3), alt.value(0)),
        tooltip=tooltip + stats_tooltip,
    ).transform_filter(
        selection_filter_pred
    ).add_params(
        chart_hover
    )

    chart_ref_line = chart_base.mark_rule(strokeDash=[2, 2]).encode(
        y=alt.datum(1. / (chart_df['bin'].max() + 1)),
        color=alt.value('#777')
    )

    return (chart_line + chart_point + chart_tooltip).properties(**properties) + chart_ref_line

def plot_diagnostics(idata, var_names=None, **kwargs):
    if var_names is None: var_names = list(idata.posterior.data_vars)

    charts = []

    for var_name in var_names:
        data_var = idata.posterior[var_name]
        legend = alt.Legend(orient='top', direction='horizontal', symbolLimit=10)
        chart_selection = chart_selection_vars(data_var)
        legend_selection = legend_selection_vars(data_var)

        chart_density = plot_density(
            data_var,
            legend=legend,
            chart_selection=chart_selection,
            legend_selection=legend_selection,
            **kwargs
        )

        chart_trace_rank = plot_rank(
            data_var,
            legend=legend,
            chart_selection=chart_selection,
            legend_selection=legend_selection,
            **kwargs
        )

        charts.append(chart_density | chart_trace_rank)

    return alt.vconcat(
        *charts
    ).resolve_scale(
        color='independent', strokeDash='independent'
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()